This notebook can be used to run the simulation instead of having to type the commands into the command line interface

In [1]:
# Copyright (c) 2021 The MATCH Authors. All rights reserved.
# Licensed under the Apache License, Version 2.0, which is in the LICENSE file.

import multiprocessing as mp
import os
from pathlib import Path
import generate_input_files
import shutil
import time

# Specify the location of your local MODEL_RUNS directory

In [2]:
# specify the location of the model runs folder you will be using, relative to the switch_model directory
# the default location is the MODEL_RUNS directory within the Git repo
# however, if you want to save model runs in a alternate location, such as a Box folder, you could use something like:
#model_runs_folder = '../../../Box/Supply/24x7 Time-Coincident Work/MODEL_RUNS'
model_runs_folder = '../MODEL_RUNS'

# enter the name of the folder within your MODEL_RUNS folder where the input and output files for this specific model run are located
model_run_name = 'solution_optimality'

model_workspace = Path.cwd() / f'{model_runs_folder}/{model_run_name}'

# Generate and Validate model inputs
1. If input files do not yet exist, create them
2. If input files exist, check that the current software version matches the version that was used to generate the inputs
3. If the versions do not match, we need to re-generate the input files to ensure consistency

In [3]:
# check if the directory exists
if os.path.exists(model_workspace / 'inputs'):
    # check that an inputs version file exists
    if os.path.exists(model_workspace / 'inputs_version.txt'):
        # get the version number
        with open(model_workspace / 'inputs_version.txt', 'r') as i:
            inputs_version = i.read()
        # get the current version number
        version_path = Path.cwd() / 'version.py'
        version = {}
        with open(version_path) as f:
            exec(f.read(), version)
        version = version['__version__']
        # check if the versions match
        if version == inputs_version:
            print(f'Input files already generated with current software version ({version})')
        else:
            print(f'Inputs were generated using version {inputs_version}. Current version is {version}')
            print('Removing old files and re-generating inputs with current version. Please wait...')
            old_files = os.listdir(model_workspace)
            old_files.remove('model_inputs.xlsx')
            for f in old_files:
                try:
                    os.remove(model_workspace / f)
                except PermissionError:
                    shutil.rmtree(model_workspace / f)
            generate_input_files.generate_inputs(model_workspace)
    else: 
        generate_input_files.generate_inputs(model_workspace)
# if the directory does not exist, generate the input files
else:
    print('Generating inputs now...')
    generate_input_files.generate_inputs(model_workspace)

Generating inputs now...
Loading data from excel spreadsheet...
Writing options.txt...
Creating input and output folders for each scenario...


C:\Users\gmiller\.conda\envs\switch_247\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Generating capacity factor timeseries for set_all_nonbasic

Starting data download for wind using 4 thread workers.
File already exists, skipping download: c:\Users\gmiller\GitHub\SWITCH247\switch_model\..\MODEL_RUNS\solution_optimality\set_all_nonbasic\PySAM Downloaded Weather Files\Wind\windtoolkit_38.1475_-121.8519_60min_80m_2012.srw

Starting data download for wind using 4 thread workers.
File already exists, skipping download: c:\Users\gmiller\GitHub\SWITCH247\switch_model\..\MODEL_RUNS\solution_optimality\set_all_nonbasic\PySAM Downloaded Weather Files\Wind\windtoolkit_38.1475_-121.8519_60min_80m_2013.srw

Starting data download for wind using 4 thread workers.
File already exists, skipping download: c:\Users\gmiller\GitHub\SWITCH247\switch_model\..\MODEL_RUNS\solution_optimality\set_all_nonbasic\PySAM Downloaded Weather Files\Wind\windtoolkit_38.1475_-121.8519_60min_80m_2014.srw

Starting data download for wind using 4 thread workers.
File already exists, skipping download: c:\U

# Solve Model Scenarios

### Identify the number of CPU cores (processors) that your computer has
If you want to solve multiple scenarios in parallel, you should only run n-1 solvers, where n is the number of CPU cores on your machine.

The following cell will actually run the scenarios. Each time you run the following cell, it will open a new parallel solver window.

In [ ]:
os.system(f'start cmd /k "cd {model_runs_folder}\{model_run_name} & activate switch_247 & switch solve-scenarios"') 

### Run as many parallel threads as possible

In [4]:
num_processors = mp.cpu_count()
print(f'This machine has {num_processors} CPU cores')
# get the number of scenarios to run
num_scenarios = len(os.listdir(model_workspace / 'inputs'))
print(f'There are {num_scenarios} scenarios to run')
# determine how many threads to open
parallel_threads = min(num_processors - 1, num_scenarios)
print(f'Running {parallel_threads} scenarios in parallel...')
i = 0
while i < parallel_threads:
    os.system(f'start cmd /k "cd {model_runs_folder}\{model_run_name} & activate switch_247 & switch solve-scenarios"') 
    time.sleep(2)
    i += 1


This machine has 28 CPU cores
There are 4 scenarios to run
Running 4 scenarios in parallel...
